In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
# Download stopwords if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load dataset
df = pd.read_csv(r"C:\Users\Dell\Desktop\Artificial Inteligence\pythonProject\dataset.csv")

In [4]:
# Function for text preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [5]:
# Apply preprocessing to text data
df["text"] = df["text"].apply(preprocess_text)

In [6]:
# Encode labels
le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])

In [7]:
# Split data into training and testing sets
text_train, text_test, target_train, target_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

In [8]:
# Ensure the text is converted to string format
text_train = [str(text) for text in text_train]
text_test = [str(text) for text in text_test]

In [9]:
# Define a pipeline with TfidfVectorizer and XGBoost
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_df=0.75, ngram_range=(1, 2))),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

In [10]:
# Perform grid search to find the best parameters
parameters = {
    'vectorizer__max_df': [0.5, 0.75, 1.0],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'classifier__max_depth': [3, 5, 7],
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.01, 0.1, 0.2]
}

In [11]:
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(text_train, target_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


C:\Users\Dell\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:57:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_df=0.75,
                                                        ngram_range=(1, 2))),
                                       ('classifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      device=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='mlogloss',
                                                      feature_types=None...
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'classifier__learning_rate': [0.01, 0.1, 0.2],
                         'classifier__max_depth': [3, 5, 7],
                         'classifier__n_estimators': [50, 100, 200],
                         'vectorizer__max_df': [0.5, 0.75, 1.0],
                         'vectorizer__ngram_range': [(1, 1), (1, 2)]},
             verbose=1)

In [12]:
# Best model from grid search
best_model = grid_search.best_estimator_

In [26]:
# Predictions
predictions_train = best_model.predict(text_train)
predictions_test = best_model.predict(text_test)

In [27]:
# Accuracy train
accuracy = accuracy_score(target_train, predictions_train)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [29]:
# Accuracy test
accuracy = accuracy_score(target_test, predictions_test)
print("Accuracy:", accuracy)

Accuracy: 0.97


In [15]:
# Function to classify new text
def classify_text(new_text, model):
    new_text = preprocess_text(new_text)
    prediction = model.predict([new_text])
    return prediction[0]

In [16]:
# Map numeric labels to original class labels
label_map = {index: label for index, label in enumerate(le.classes_)}

In [17]:
# Classify new texts
new_text1 = "Our agency excels in delivering real business value through data science"
prediction1 = classify_text(new_text1, best_model)
print("Prediction for new_text1:", label_map[prediction1])

Prediction for new_text1: Agencies


In [18]:
new_text2 = "Any recommendations for top data science conferences or meetups?"
prediction2 = classify_text(new_text2, best_model)
print("Prediction for new_text2:", label_map[prediction2])

Prediction for new_text2: Students


In [23]:
new_text6 = "our highly trained staff, they are the so great they can do anything, they are best in the business"                                           # best
prediction6 = classify_text(new_text6, best_model)
print("Prediction for new_text6:", label_map[prediction6])

Prediction for new_text6: Agencies
